# Metryki klasyfikacji

Aby poznać popularne metody oceny klasyfikatorów wygenerujemy sobie przykładowe sekwencje klas przykładów. Wykorzystamy w tym celu bibliotekę numpy oraz generator liczb losowych z rozkładu normalnego.

In [1]:
import numpy as np
from numpy.random import RandomState

In [2]:
random = RandomState(30)

random_1 = random.normal(loc = 0.0, size = 100)
random_2 = random.logistic(loc = 1, size = 100)

In [5]:
print(random_1[:10])
print(random_2[:10])

[-1.26405266  1.52790535 -0.97071094  0.47055962 -0.10069672  0.30379318
 -1.72596243  1.58509537  0.13429659 -1.10685547]
[-0.34625296  2.7465497   0.46063247  3.2000744   0.12353606  1.56724961
  2.32137633 -0.21323077  1.9572528   0.03456367]


In [6]:
y_test = [1 if i >= 0 else 0 for i in random_1]
y_pred = [1 if i >= 0 else 0 for i in random_2]

In [9]:
print(y_test[:10])
print(y_pred[:10])

[0, 1, 0, 1, 0, 1, 0, 1, 1, 0]
[0, 1, 1, 1, 1, 1, 1, 0, 1, 1]


## Macierz błędów

Pierwszym krokiem będzie stworzenie macierzy błędów, czyli wyznaczenia true postives, true negatives, false positives i false negatives.

In [16]:
def confusion_matrix(truth, prediction):
    tp, tn, fp, fn = 0, 0, 0, 0
    
    for label_true, label_pred in zip(truth, prediction):
        if label_true == label_pred:
            if label_true == 1:
                tp += 1
            else:
                tn += 1
        else:
            if label_pred == 1:
                fp += 1
            else:
                fn += 1
    return tp, tn, fp, fn

In [17]:
tp, tn, fp, fn = confusion_matrix(y_test, y_pred)

In [18]:
print("True positive: ", tp, "\nTrue Negative: ", tn, "\nFalse Positive: ", fp, "\nFalse Negative: ", fn)

True positive:  30 
True Negative:  14 
False Positive:  43 
False Negative:  13


## Accuracy

Pierwszą miarą jest miara dokładności 

accuracy = (tp+tn) / (tp+tn+fp+fn)

In [42]:
def accuracy(truth, prediction):
    tp, tn, fp, fn = confusion_matrix(truth, prediction)
    
    return (tp+tn) / (tp+tn+fp+fn)

In [43]:
accuracy(y_test, y_pred)

0.44

Wynik dla analizowanych predykcji

#### Problem
Accuracy zachowuje się źle przy niezbalansowanych zbiorach. Wygenerujemy zbiór, w którym większość przykładów będzie negatywna.

In [44]:
random_3 = random.normal(loc = -1.5, size=100)

In [45]:
y_pred_2 = [1 if i >= 0 else 0 for i in random_3]

In [46]:
sum(y_pred_2)

6

In [47]:
accuracy(y_test, y_pred_2)

0.57

Zakładając z góry, że wszystkie przykłady są negatywne uzyskujemy bardzo wysoką dokładność. Takie zachowanie jest w wielu przypadkach niepożądane - przypuśćmy, że próbujemy stworzyć klasyfikator do komórek rakowych, gdzie większość przypadków jest negatywna - dla takiego podejścia, zwrócenie informacji że wszystkie przypadki są negatywne da bardzo wysoką dokładność.

In [49]:
accuracy(100*[0], y_pred_2)

0.94

0

## Recall

Ta miara z kolei mówi o tym, jaką część dodatnich wyników wykrył klasyfikator.

recall = tp / (tp + fn)

In [62]:
def recall(truth, prediction):
    tp, tn, fp, fn = confusion_matrix(truth, prediction)
    
    return round(tp / (tp + fn), 4)

In [63]:
recall(y_test, y_pred)

0.6977

In [64]:
recall(y_test, y_pred_2)

0.0698

## Precision

Jest to miara, która skupia się tylko na przykładach pozytywnych - mówi jaka część wyników wskazanych przez klasyfikator jako dodatnie jest rzeczywiście dodatnia.

precision = tp / (tp+fp)

In [66]:
def precision(truth, prediction):
    tp, tn, fp, fn = confusion_matrix(truth, prediction)
    
    return round(tp / (tp+fp), 4)

In [67]:
precision(y_test, y_pred)

0.411

In [68]:
precision(y_test, y_pred_2)

0.5

# F1 Score

Jest to średnia harmoniczna precyzji i czułości. Ogólnie - im wyższy F1-score tym lepszy jest klasyfikator.

f_score = (2 * precision * recall)/(prec+rec)

In [73]:
def f_score(truth, prediction):
    tp, tn, fp, fn = confusion_matrix(truth, prediction)

    
    recall = tp / (tp + fn)
    precision = tp / (tp+fp)
    
    return (2 * precision * recall)/(precision+recall)

In [74]:
f_score(y_test, y_pred)

0.5172413793103448

In [75]:
f_score(y_test, y_pred_2)

0.12244897959183673

**Zadanie:** Sprawdź funckje f1_score, precision_score, recall_score z modułu scikit learn

In [85]:
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report

In [86]:
f1_score(y_test, y_pred)

0.5172413793103448

In [87]:
f1_score(y_test, y_pred_2)

0.12244897959183673

In [88]:
precision_score(y_test, y_pred)

0.410958904109589

In [89]:
precision_score(y_test, y_pred)

0.410958904109589

In [90]:
recall_score(y_test, y_pred)

0.6976744186046512

In [91]:
recall_score(y_test, y_pred_2)

0.06976744186046512

In [93]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.52      0.25      0.33        57
           1       0.41      0.70      0.52        43

    accuracy                           0.44       100
   macro avg       0.46      0.47      0.43       100
weighted avg       0.47      0.44      0.41       100

